이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 FastText 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/22883  

이 자료는 2023년 10월 18일에 마지막으로 테스트되었습니다.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
pip list | grep gensim

gensim                           4.3.2


In [3]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7c7b6c66b9d0>)

In [5]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [6]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [7]:
from gensim.models import Word2Vec, FastText

In [9]:
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [10]:
# 입력 단어에 대해서 유사한 단어를 찾아내는 코드에 electrofishing이라는 단어를 넣어봅니다.
model.wv.most_similar("electrofishing")

# 에러 메시지는 단어 집합(Vocabulary)에 electrofishing이 존재하지 않는다고 합니다.
# 이처럼 Word2Vec는 학습 데이터에 존재하지 않는 단어.
# 즉, 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않기 때문에 단어의 유사도를 계산할 수 없습니다.

KeyError: ignored

In [11]:
# 실행시간 3분 정도 소요
model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [12]:
# Word2Vec은 학습하지 않은 단어에 대해서 유사한 단어를 찾아내지 못했지만,
# FastText는 유사한 단어를 계산해서 출력하고 있음을 볼 수 있습니다.

model.wv.most_similar("electrofishing")

[('electrolux', 0.8756399154663086),
 ('electrolyte', 0.8665815591812134),
 ('electro', 0.8555654883384705),
 ('electroshock', 0.8541486859321594),
 ('electroencephalogram', 0.8421139717102051),
 ('electrochemical', 0.8332715034484863),
 ('electrogram', 0.8332095146179199),
 ('electric', 0.8262913823127747),
 ('electrode', 0.8210142254829407),
 ('electron', 0.8209858536720276)]